In [ ]:
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS, NBEATSx, PatchTST, iTransformer, TiDE, TSMixer, TFT
from neuralforecast.losses.pytorch import *
from neuralforecast.tsdataset import TimeSeriesDataset

warnings.filterwarnings("ignore")
os.environ["NIXTLA_ID_AS_COL"] = "true"
pd.set_option('display.precision', 3)

In [ ]:
data_url = 'https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/master/data/PJM_Load_hourly.csv'
df = pd.read_csv(data_url, parse_dates=['Datetime'])
df.columns = ['ds', 'y']
df.insert(0, 'unique_id', 'PJM')
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)
df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-700:], df['y'][-700:])
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
Y_train_df = df[:-96]
Y_test_df = df[-96:]

horizon = 96

## PatchTST

In [ ]:
model = PatchTST(h=horizon,
                 input_size=2*horizon,
                 # PatchTST params
                 scaler_type='robust',
                 loss=MAE(),
                 max_steps=500,
                 val_check_steps=50,
                 early_stop_patience_steps=2)

nf = NeuralForecast(
    models=[model],
    freq='H'
)

nf.fit(Y_train_df, val_size=horizon)
patchtst_preds = nf.predict()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-200:], df['y'][-200:])
ax.plot(patchtst_preds['ds'], patchtst_preds['PatchTST'], ls='--', label='PatchTST')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
Y_test_df['PatchTST'] = patchtst_preds['PatchTST'].values
Y_test_df.head()

## iTransformer

In [ ]:
model = iTransformer(h=horizon,
                     input_size=2*horizon,
                     # iTransformer params
                     loss=MSE(),
                     valid_loss=MAE(),
                     early_stop_patience_steps=3,
                     scaler_type='identity',
                     batch_size=32)

nf = NeuralForecast(
    models=[model],
    freq='H'
)

nf.fit(Y_train_df, val_size=horizon)
iTransformer_preds = nf.predict()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-200:], df['y'][-200:])
ax.plot(iTransformer_preds['ds'], iTransformer_preds['iTransformer'], ls='--', label='iTransformer')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
Y_test_df['iTransformer'] = iTransformer_preds['iTransformer'].values

## TiDE

In [ ]:
model = TiDE(h=horizon,
             input_size=2*horizon,
             # TiDE params
             loss=MAE(),
             valid_loss=MAE(),
             early_stop_patience_steps=3,
             scaler_type='identity',
             batch_size=32)

nf = NeuralForecast(
    models=[model],
    freq='H'
)

nf.fit(Y_train_df, val_size=horizon)
tide_preds = nf.predict()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-200:], df['y'][-200:])
ax.plot(tide_preds['ds'], tide_preds['TiDE'], ls='--', label='TiDE')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
Y_test_df['TiDE'] = tide_preds['TiDE'].values
Y_test_df.head()

## TSMixer

In [ ]:
model = TSMixer(h=horizon,
                input_size=2*horizon,
                # TSMixer params
                scaler_type='standard',
                max_steps=500,
                early_stop_patience_steps=-1,
                val_check_steps=5,
                learning_rate=1e-3,
                loss=MSE(),
                valid_loss=MAE(),
                batch_size=32
                )

nf = NeuralForecast(
    models=[model],
    freq='H'
)

nf.fit(Y_train_df, val_size=horizon)
tsmixer_preds = nf.predict()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-200:], df['y'][-200:])
ax.plot(tsmixer_preds['ds'], tsmixer_preds['TSMixer'], ls='--', label='TSMixer')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
Y_test_df['TSMixer'] = tsmixer_preds['TSMixer'].values
Y_test_df.head()

In [ ]:
evaluation = evaluate(df=Y_test_df, metrics=[mae, smape])
evaluation